In [5]:
# Installing Required Libraries
%pip install python-docx
%pip install python-pptx
%pip install PyPDF2
%pip install langchain
%pip install langchain_community
%pip install langchain_google_genai
%pip install langchain_text_splitters
%pip install sentence-transformers
%pip install faiss-cpu
%pip install cohere

In [4]:
from PyPDF2 import PdfReader
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [7]:
pdf_file = open('/content/Data Cleaning -1.pdf','rb')

In [8]:
pdf_text = ""
pdf_reader = PdfReader(pdf_file)
for page in pdf_reader.pages:
    pdf_text += page.extract_text()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len,
        separators=['\n', '\n\n', ' ', '']
    )

chunks = text_splitter.split_text(text =pdf_text)

In [10]:
len(chunks)

26

In [11]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/tmp/ipython-input-3701169180.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

In [12]:
%pip install faiss-cpu

In [13]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [14]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [15]:
retrieved_docs = retriever.invoke("WHAT IS DATA CLEANING")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [16]:
len(retrieved_docs)

6

In [17]:
print(retrieved_docs[0].page_content)

DATA CLEANING
Transform Raw DATA into Actionable Insights Using SQL
Created By: Muhammad Umar Hanif
WHAT IS DAT A CLEANING?
Data cleaning in SQL involves identifying and correcting errors or inconsistencies in data to 
improve its quality and accuracy. It includes tasks like:
This process ensures that the data is clean, consistent, and ready for analysis.
01. Handle Missing Values:
The SQL functions 
•COALESCE()
•IFNULL()
•ISNULL() 
are used to handle missing or NULL values in databases. Let's explore how these functions work with a 
common example.
1.1 - COALESCE(): 
Returns the first non- NULL value from the list of arguments.
Example:
Suppose you have a table called employees with columns for base_salary, bonus, and 
total_compensation , where some values in bonus and total_compensation  might be NULL.
idnamebase_salarybonustotal_compensation
1John50000NULLNULL
2Sarah600005000NULL
3David45000NULLNULL
4Alice70000700077000
Case 01:
SELECT name, 
       base_salary,


In [18]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                not contained in the context, say "answer not available in context" \n\n
                Context: \n {context}?\n
                Question: \n {question} \n
                Answer:"""

prompt = PromptTemplate.from_template(template=prompt_template)

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [25]:
def generate_answer(question):
    cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key = '01XCVjqwF2sJPaSB2e2GRCNeGZVVMJiNxJyFpktR')

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | cohere_llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

In [26]:
ans = generate_answer("Methods to handle missing values?")
print(ans)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 The provided context mentions two methods to handle missing values, COALESCE() and IFNULL(). COALESCE() returns the first non-NULL value from a list of arguments, while IFNULL() replaces NULL with a specified value. These functions are used to handle missing or NULL values in databases, ensuring data integrity and consistency for further analysis.


In [27]:
ans = generate_answer("What's Mathmatics")
print(ans)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 The answer not available in the provided context. 

Do you have a specific mathematical equation you'd like help with? 
